**IMPORTANT**: Before starting this notebook make sure that the kernel of the previous notebook is shutdown or reset it's state to forget the previous `model_user` Nillion client

In [1]:
## If problems arise with the loading of the shared library, this script can be used to load the shared library before other libraries.
## Remember to also run on your local machine the script below:
# bash replace_lib_version.sh

import platform
import ctypes

if platform.system() == "Linux":
    # Force libgomp and py_nillion_client to be loaded before other libraries consuming dynamic TLS (to avoid running out of STATIC_TLS)
    ctypes.cdll.LoadLibrary("libgomp.so.1")
    ctypes.cdll.LoadLibrary(
        "/home/vscode/.local/lib/python3.12/site-packages/py_nillion_client/py_nillion_client.abi3.so"
    )

In [2]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import math

import json
import joblib

from common.utils import compute, store_secret_array

from dotenv import load_dotenv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import nada_numpy as na
import nada_numpy.client as na_client
import py_nillion_client as nillion
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey
from py_nillion_client import NodeKey, UserKey
from nillion_python_helpers import (create_nillion_client,
                                    create_payments_config)


home = os.getenv("HOME")
load_dotenv(f"{home}/.config/nillion/nillion-devnet.env")

True

## Authenticate with Nillion

To connect to the Nillion network, we need to have a user key and a node key. These serve different purposes:

The `user_key` is the user's private key. The user key should never be shared publicly, as it unlocks access and permissions to secrets stored on the network.

The `node_key` is the node's private key which is run locally to connect to the network.

In [3]:
# Load all Nillion network environment variables
assert os.getcwd().endswith(
    "examples/spam_detection"
), "Please run this script from the examples/spam_detection directory otherwise, the rest of the tutorial may not work"
load_dotenv()

True

In [4]:
cluster_id = os.getenv("NILLION_CLUSTER_ID")
grpc_endpoint = os.getenv("NILLION_NILCHAIN_GRPC")
chain_id = os.getenv("NILLION_NILCHAIN_CHAIN_ID")
seed = "my_seed"
model_user_userkey = UserKey.from_seed((seed))
model_user_nodekey = NodeKey.from_seed((seed))
model_user_client = create_nillion_client(model_user_userkey, model_user_nodekey)
model_user_party_id = model_user_client.party_id
model_user_user_id = model_user_client.user_id

In [5]:
payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

In [6]:
# This information was provided by the model provider
with open("target/tmp.json", "r") as provider_variables_file:
    provider_variables = json.load(provider_variables_file)

program_id = provider_variables["program_id"]
model_store_id = provider_variables["model_store_id"]
model_provider_party_id = provider_variables["model_provider_party_id"]

print("Program ID: ", program_id)
print("Model Store ID: ", model_store_id)
print("Model Provider Party ID: ", model_provider_party_id)

Program ID:  3rgqxWd47e171EUwe4RXP9hm45tmoXfuF8fC52S7jcFoQTnU8wPiL7hqWzyV1muak6bEg7iWhudwg4t2pM9XnXcp/spam_detection
Model Store ID:  1a67209f-359e-46fa-9451-6dd559c27ca1
Model Provider Party ID:  12D3KooWJHrXiK2JTCjJxwCCktJPSYsUsz2WHEBSB5iZtqGiZ8Qm


## Model user flow

### Convert text to features

In [7]:
vectorizer: TfidfVectorizer = joblib.load("model/vectorizer.joblib")

In [8]:
# Let's find out whether it's a billion dollar opportunity or pyramid scheme
INPUT_DATA = "Free entry in 2 a wkly comp to win exclusive prizes! Text WIN to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

[features] = vectorizer.transform([INPUT_DATA]).toarray().tolist()

In [9]:
features = np.array(features).astype(float)

### Send features to Nillion

In [10]:
permissions = nillion.Permissions.default_for_user(model_user_client.user_id)
permissions.add_compute_permissions({model_user_client.user_id: {program_id}})

images_store_id = await store_secret_array(
    model_user_client,
    payments_wallet,
    payments_client,
    cluster_id,
    features,
    "my_input",
    na.SecretRational,
    1,
    permissions,
)

Getting quote for operation...
Quote cost is 48002 unil
Submitting payment receipt 48002 unil, tx hash ADC46091FE92101AC9740304E5C2DF883C90ACF66EB155EAEFE23D4438479A4A


### Run inference & check result

In [11]:
compute_bindings = nillion.ProgramBindings(program_id)

compute_bindings.add_input_party("Provider", model_provider_party_id)
compute_bindings.add_input_party("User", model_user_party_id)
compute_bindings.add_output_party("User", model_user_party_id)

In [12]:
result = await compute(
    model_user_client,
    payments_wallet,
    payments_client,
    program_id,
    cluster_id,
    compute_bindings,
    [model_store_id, images_store_id],
    nillion.NadaValues({}),
    verbose=True,
)

logit = na_client.float_from_rational(result["logit_0"])
print("Computed logit is", logit)

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

output_probability = sigmoid(logit)

print("Which corresponds to probability", output_probability)

Getting quote for operation...
Quote cost is 1004 unil
Submitting payment receipt 1004 unil, tx hash 3B0F0A512E3459077F4B0C76C51573FE6E0123695BA5350C88C82B74D4ED7799
✅ Compute complete for compute_id d256e8bb-b85a-428f-b168-6359cfaf3ec8
🖥️  The result is {'logit_0': 157897}
Computed logit is 2.4093170166015625
Which corresponds to probability 0.9175350190040187


### Compare result to what we would have gotten in plain-text inference

In [13]:
vectorizer: TfidfVectorizer = joblib.load("model/vectorizer.joblib")
classifier: LogisticRegression = joblib.load("model/classifier.joblib")

In [14]:
features = vectorizer.transform([INPUT_DATA]).toarray().tolist()

In [15]:
[logit_plain_text] = classifier.decision_function(features)

In [16]:
print("Logit in plain text: {}".format(logit_plain_text))

Logit in plain text: 2.408079563074276


In [17]:
[result] = classifier.predict_proba(features)
output_probability_plain_text = result[1]

In [18]:
print(
    "Probability of spam in plain text: {:.6f}%".format(
        output_probability_plain_text * 100
    )
)
print("Probability of spam in Nillion: {:.6f}%".format(output_probability * 100))

Probability of spam in plain text: 91.744134%
Probability of spam in Nillion: 91.753502%
